In [1]:
import json
import numpy as np
import operator
import math

In [2]:
def r_precision(G, R):
    limit_R = R[:len(G)]
    if len(G) != 0:
        return len(list(set(G).intersection(set(limit_R)))) * 1.0 / len(G)
    else:
        return 0

def ndcg(G, R):
    r = [1 if i in set(G) else 0 for i in R]
    r = np.asfarray(r)
    dcg = r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
    #k = len(set(G).intersection(set(R)))
    k = len(G)
    if k > 0:
        idcg = 1 + np.sum(np.ones(k - 1) / np.log2(np.arange(2, k + 1)))
        return dcg * 1.0 / idcg
    else:
        return 0

def clicks(G, R):
    n = 1
    for i in R:
        if i in set(G):
            return ((n - 1) / 10) * 1.0
        n += 1
    return 51

In [3]:
GT = json.load(open('../DATA_PROCESSING/PL_TRACKS_ALL.json'))
# GT = json.load(open('../DATA_PROCESSING/PL_TRACKS_5_TEST.json'))
track_index = json.load(open('../DATA_PROCESSING/ALL_INDEX_READONLY/TRACK_INDEX_READONLY.json'))
track_index_reversed = {k:v for (v, k) in track_index.items()}

In [479]:
Task = str(5)

In [480]:
if Task == '1':
    bar = 1
elif Task == '2':
    bar = 5
elif Task == '5':
    bar = 10
elif Task == '6':
    bar = 25
elif Task == '7':
    bar = 25
elif Task == '9':
    bar = 100

## For different Task (change from here)

In [481]:
SEED = json.load(open('../DATA_PROCESSING/PL_TRACKS_5_TEST_T' + Task + '.json'))

In [482]:
GT_Now = {}
for pl in SEED:
    GT_Now[pl] = []
    for t in GT[pl]:
        if t not in SEED[pl]:
            GT_Now[pl].append(t)

In [483]:
Xing_Raw = json.load(open('../MODEL_5_Counter_Artist_Album/Pure_Xing/Pure_Xing_T' + Task + '.json'))
Xing = {}
for pl in Xing_Raw:
    Xing[pl] = []
    for t in Xing_Raw[pl]:
        Xing[pl].append(track_index_reversed[t])

In [484]:
QQ = json.load(open('../MODEL_5_Counter_Artist_Album/Pure_QQ/T' + Task + '_100_500_50.json'))

In [485]:
W2V_raw = json.load(open('/home/xing/xing/xing/T' + Task + '.json'))

In [486]:
# W2V = {}
# for pl in W2V_raw:
#     W2V[pl] = []
#     for t in W2V_raw[pl]:
#         if t not in GT[pl]:
#             W2V[pl].append(t)
#         if len(W2V[pl]) == 500:
#             break
            
            
W2V = {}
for pl in W2V_raw:
    W2V[pl] = []
    for t in W2V_raw[pl]:
        if t not in SEED[pl]:
            W2V[pl].append(t)
        if len(W2V[pl]) == 500:
            break
    

## Combine Method 1 (Used in current submission)

In [487]:
def combine_method_1(QQ_dict, Xing_dict):
    R_combined = {}
    for pl in Xing_dict:
        if len(Xing_dict[pl]) == 500:
            R_combined[pl] = Xing_dict[pl][:]
        else:
            rest = 500 - len(Xing_dict[pl])
            R_combined[pl] = Xing_dict[pl][:]
            for t in QQ_dict[pl]:
                if t not in R_combined[pl]:
                    R_combined[pl].append(t)
                    
                if len(R_combined[pl]) == 500:
                    break
    return R_combined

In [488]:
Combined_R_method_1 = combine_method_1(QQ, Xing)

In [489]:
R_precision_1 = {}
NDCG_1 = {}
Clicks_1 = {}

for pl in Xing:

    R_precision_1[pl] = r_precision(GT_Now[pl], Combined_R_method_1[pl])
    NDCG_1[pl] = ndcg(GT_Now[pl], Combined_R_method_1[pl])
    Clicks_1[pl] = clicks(GT_Now[pl], Combined_R_method_1[pl])
    
print sum(R_precision_1.values()) / len(R_precision_1)
print sum(NDCG_1.values()) / len(NDCG_1)
print sum(Clicks_1.values()) / len(Clicks_1)

0.202342210441
0.3833401973331007
1.5


## Combine Method 2

In [504]:
def rank(new, current, sort_base):
    layer = list(set(new) - set(current))
    layer_index = {}
    for elem in layer:
        layer_index[elem] = sort_base[elem]
        
    layer_sorted = sorted(layer_index.items(), key=operator.itemgetter(1))
    layer_final = [i[0] for i in layer_sorted]
    return current + layer_final

def combine_method_2_sub_1(QQ, Xing, GT_len):
    QQ_index = {}
    for elem in QQ:
        QQ_index[elem] = len(QQ_index)
    
    Xing_index = {}
    for elem in Xing:
        Xing_index[elem] = len(Xing_index)
        
        
    Ax = set(Xing[:GT_len])
    Bx = set(Xing[GT_len:])
        
    Aq = set(QQ[:GT_len])
    Bq = set(QQ[GT_len:])
    
    # Layer 1
    up_to_layer1 = rank(list(Ax & Aq), [], Xing_index)
    
    # Layer 2 part 1
    up_to_layer2_1 = rank(list(Ax & Bq), up_to_layer1, Xing_index)

    # Layer 2 part 2
    up_to_layer2_2 = rank(list(Bx & Aq), up_to_layer2_1, Xing_index)
    
    # Layer 3 part 1
    up_to_layer3_1 = rank(list(Ax), up_to_layer2_2, Xing_index)
    
    # Layer 3 part 2
    up_to_layer3_2 = rank(list(Aq), up_to_layer3_1, QQ_index)
    
    # Layer 3 part 3
    up_to_layer3_3 = rank(list(Bx & Bq), up_to_layer3_2, Xing_index)
    
    # Layer 4 part 1
    up_to_layer4_1 = rank(list(Bx), up_to_layer3_3, Xing_index)
    
    # Layer 4 part 2
    up_to_layer4_2 = rank(list(Bq), up_to_layer4_1, QQ_index)
    
    return up_to_layer4_2[:500]

def combine_method_2_sub_2(QQ, Xing, GT_len):
    QQ_index = {}
    for elem in QQ:
        QQ_index[elem] = len(QQ_index)
    
    Xing_index = {}
    for elem in Xing:
        Xing_index[elem] = len(Xing_index)
        
        
    Ax = set(Xing[:GT_len])
    Bx = set(Xing[GT_len:])
        
    Aq = set(QQ[:GT_len])
    Bq = set(QQ[GT_len:])
    
    # Layer 1
    up_to_layer1 = rank(list(Ax & Aq), [], Xing_index)
    
    # Layer 2 part 1
    up_to_layer2_1 = rank(list(Ax & Bq), up_to_layer1, Xing_index)


    # Layer 2 part 2
    up_to_layer2_2 = rank(list(Ax), up_to_layer2_1, Xing_index)
    
    # Layer 3 part 1
    up_to_layer3_1 = rank(list(Bx & Aq), up_to_layer2_2, Xing_index)
    
    # Layer 3 part 2
    up_to_layer3_2 = rank(list(Aq), up_to_layer3_1, QQ_index)
    
    # Layer 3 part 3
    up_to_layer3_3 = rank(list(Bx & Bq), up_to_layer3_2, Xing_index)
    
    # Layer 4 part 1
    up_to_layer4_1 = rank(list(Bx), up_to_layer3_3, Xing_index)
    
    # Layer 4 part 2
    up_to_layer4_2 = rank(list(Bq), up_to_layer4_1, QQ_index)
    
    return up_to_layer4_2[:500]

def combine_method_2_sub_3(QQ, Xing, GT_len):
    QQ_index = {}
    for elem in QQ:
        QQ_index[elem] = len(QQ_index)
    
    Xing_index = {}
    for elem in Xing:
        Xing_index[elem] = len(Xing_index)
        
        
    Ax = set(Xing[:GT_len])
    Bx = set(Xing[GT_len:])
        
    Aq = set(QQ[:GT_len])
    Bq = set(QQ[GT_len:])
    
    # Layer 1
    up_to_layer1 = rank(list(Ax & Aq), [], Xing_index)
    

    # Layer 2 part 2
    up_to_layer2_1 = rank(list(Bx & Aq), up_to_layer1, Xing_index)
    
    # Layer 2 part 1
    up_to_layer2_2 = rank(list(Ax & Bq), up_to_layer2_1, Xing_index)
    
    # Layer 3 part 1
    up_to_layer3_1 = rank(list(Ax), up_to_layer2_2, Xing_index)
    
    # Layer 3 part 2
    up_to_layer3_2 = rank(list(Aq), up_to_layer3_1, QQ_index)
    
    # Layer 3 part 3
    up_to_layer3_3 = rank(list(Bx & Bq), up_to_layer3_2, Xing_index)
    
    # Layer 4 part 1
    up_to_layer4_1 = rank(list(Bx), up_to_layer3_3, Xing_index)
    
    # Layer 4 part 2
    up_to_layer4_2 = rank(list(Bq), up_to_layer4_1, QQ_index)
    
    return up_to_layer4_2[:500]

def combine_method_2_sub_4(QQ, Xing, GT_len):
    QQ_index = {}
    for elem in QQ:
        QQ_index[elem] = len(QQ_index) * 1.5
    
    Xing_index = {}
    for elem in Xing:
        Xing_index[elem] = len(Xing_index) * 1.0 
        
    final_index = {}
    for elem in list(set(Xing_index.keys()) | set(QQ_index.keys())):
        if elem in Xing and elem not in QQ:
            final_index[elem] = Xing_index[elem]
        elif elem in QQ and elem not in Xing:
            final_index[elem] = QQ_index[elem]
        else:
            final_index[elem] = Xing_index[elem] + QQ_index[elem]
        
        
    Ax = set(Xing[:GT_len])
    Bx = set(Xing[GT_len:])
        
    Aq = set(QQ[:GT_len])
    Bq = set(QQ[GT_len:])
    
    # Layer 1
    up_to_layer1 = rank(list(Ax & Aq), [], final_index)
    
    # Layer 2 part 1
    up_to_layer2_1 = rank(list(Ax & Bq), up_to_layer1, final_index)

    # Layer 2 part 2
    up_to_layer2_2 = rank(list(Bx & Aq), up_to_layer2_1, final_index)
    
    # Layer 3 part 1
    up_to_layer3_1 = rank(list(Ax), up_to_layer2_2, final_index)
    
    # Layer 3 part 2
    up_to_layer3_2 = rank(list(Aq), up_to_layer3_1, final_index)
    
    # Layer 3 part 3
    up_to_layer3_3 = rank(list(Bx & Bq), up_to_layer3_2, final_index)
    
    # Layer 4 part 1
    up_to_layer4_1 = rank(list(Bx), up_to_layer3_3, final_index)
    
    # Layer 4 part 2
    up_to_layer4_2 = rank(list(Bq), up_to_layer4_1, final_index)
    
    return up_to_layer4_2[:500]

def combine_method_2_sub_5(QQ, Xing, GT_len):
    QQ_index = {}
    for elem in QQ:
        QQ_index[elem] = (500 - len(QQ_index)) * 2.0
    
    Xing_index = {}
    for elem in Xing:
        Xing_index[elem] = (500 - len(Xing_index)) * 3.0
        
    final_index = {}
    for elem in list(set(Xing_index.keys()) | set(QQ_index.keys())):
        if elem in Xing and elem not in QQ:
            final_index[elem] = Xing_index[elem]
        elif elem in QQ and elem not in Xing:
            final_index[elem] = QQ_index[elem]
        else:
            final_index[elem] = Xing_index[elem] + QQ_index[elem]
            
    layer_sorted = sorted(final_index.items(), key=operator.itemgetter(1), reverse = True)
    layer_final = [i[0] for i in layer_sorted] 
    
    return layer_final[:500]

def combine_method_2_sub_6(QQ, Xing, W2V, GT_len):
    
    
    all_tracks = list(set(QQ + Xing + W2V))
    QQ_index = {}
    t = 0
    for elem in QQ:
        QQ_index[elem] = (501 - t) * 2.0
        t += 1
    
    Xing_index = {}
    t = 0
    for elem in Xing:
        Xing_index[elem] = (501 - t) * 3.0
        t += 1
        
    W2V_index = {}
    t = 0
    for elem in W2V:
        W2V_index[elem] = (501 - t) * 1.5
        t += 1
            
    for elem in all_tracks:
        if elem not in QQ_index:
            QQ_index[elem] = 0
        if elem not in Xing_index:
            Xing_index[elem] = 0
        if elem not in W2V_index:
            W2V_index[elem] = 0
        
    final_index = {}
    for elem in all_tracks:
        
        final_index[elem] = Xing_index[elem] + QQ_index[elem]  + W2V_index[elem]
            
    layer_sorted = sorted(final_index.items(), key=operator.itemgetter(1), reverse = True)
    layer_final = [i[0] for i in layer_sorted] 
    
    return layer_final[:500]
        
        
#     Ax = set(Xing[:GT_len])
#     Bx = set(Xing[GT_len:])
        
#     Aq = set(QQ[:GT_len])
#     Bq = set(QQ[GT_len:])
    
#     # Layer 1
#     up_to_layer1 = rank(list(Ax & Aq), [], final_index)
    
#     # Layer 2 part 1
#     up_to_layer2_1 = rank(list(Ax & Bq), up_to_layer1, final_index)

#     # Layer 2 part 2
#     up_to_layer2_2 = rank(list(Bx & Aq), up_to_layer2_1, final_index)
    
#     # Layer 3 part 1
#     up_to_layer3_1 = rank(list(Ax), up_to_layer2_2, final_index)
    
#     # Layer 3 part 2
#     up_to_layer3_2 = rank(list(Aq), up_to_layer3_1, final_index)
    
#     # Layer 3 part 3
#     up_to_layer3_3 = rank(list(Bx & Bq), up_to_layer3_2, final_index)
    
#     # Layer 4 part 1
#     up_to_layer4_1 = rank(list(Bx), up_to_layer3_3, final_index)
    
#     # Layer 4 part 2
#     up_to_layer4_2 = rank(list(Bq), up_to_layer4_1, final_index)
    
#    return up_to_layer4_2[:500]

def combine_method_2(QQ_dict, Xing_dict, W2V_dict):
    global GT
    R_combined = {}
    for pl in Xing_dict:
        R_combined[pl] = combine_method_2_sub_6(QQ_dict[pl], Xing_dict[pl], W2V_dict[pl], len(GT_Now[pl]))
    return R_combined

In [505]:
Combined_R_method_2 = combine_method_2(QQ, Xing, W2V)

In [506]:
R_precision_2 = {}
NDCG_2 = {}
Clicks_2 = {}

for pl in Xing:
    R_precision_2[pl] = r_precision(GT_Now[pl], Combined_R_method_2[pl])
    NDCG_2[pl] = ndcg(GT_Now[pl], Combined_R_method_2[pl])
    Clicks_2[pl] = clicks(GT_Now[pl], Combined_R_method_2[pl])

print sum(R_precision_2.values()) / len(R_precision_2)
print sum(NDCG_2.values()) / len(NDCG_2)
print sum(Clicks_2.values()) / len(Clicks_2)

0.208497709966
0.3911172168730489
1.142


In [502]:
0.217464854917
0.4193948692135327
0.722

0.722

In [503]:
R_precision_2 = {}
NDCG_2 = {}
Clicks_2 = {}

for pl in W2V:
    R_precision_2[pl] = r_precision(GT_Now[pl], W2V[pl])
    NDCG_2[pl] = ndcg(GT_Now[pl], W2V[pl])
    Clicks_2[pl] = clicks(GT_Now[pl], W2V[pl])

print sum(R_precision_2.values()) / len(R_precision_2)
print sum(NDCG_2.values()) / len(NDCG_2)
print sum(Clicks_2.values()) / len(Clicks_2)

0.113440726478
0.21861119536644122
4.34
